Data source [from kaggle.com](https://www.kaggle.com/dorbicycle/world-foodfeed-production)  
World food consumption trends [article on FAO.org](http://www.fao.org/docrep/005/ac911e/ac911e05.htm)

In [401]:
## Importing Necessary Libraries
import pandas as pd
import numpy as np

In [402]:
raw_df = pd.read_csv(r'C:\Users\pooja\Desktop\FAO.csv', encoding='latin1')

In [403]:
# Make a copy of original dataframe
df = raw_df.copy()

In [404]:
print(f"Number of Observations in df:{df.shape}")
df.head()

Number of Observations in df:(21477, 63)


,Area Abbreviation,Area Code,Area,Item Code,Item,Element Code,Element,Unit,latitude,longitude,...,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013
0,AFG,2,Afghanistan,2511,Wheat and products,5142,Food,1000 tonnes,33.94,67.71,...,3249.0,3486.0,3704.0,4164.0,4252.0,4538.0,4605.0,4711.0,4810,4895
1,AFG,2,Afghanistan,2805,Rice (Milled Equivalent),5142,Food,1000 tonnes,33.94,67.71,...,419.0,445.0,546.0,455.0,490.0,415.0,442.0,476.0,425,422
2,AFG,2,Afghanistan,2513,Barley and products,5521,Feed,1000 tonnes,33.94,67.71,...,58.0,236.0,262.0,263.0,230.0,379.0,315.0,203.0,367,360
3,AFG,2,Afghanistan,2513,Barley and products,5142,Food,1000 tonnes,33.94,67.71,...,185.0,43.0,44.0,48.0,62.0,55.0,60.0,72.0,78,89
4,AFG,2,Afghanistan,2514,Maize and products,5521,Feed,1000 tonnes,33.94,67.71,...,120.0,208.0,233.0,249.0,247.0,195.0,178.0,191.0,200,200


## Assesment
### Tidiness
* Year columns showing the quantity of food and feed consumption to be melted
* Unnessesary columns removed
### Quality
* 'Y' before the years to be removed
* `Item` values to be further grouped into categories

In [405]:
print(f"Number of Countries:{df['Area'].nunique()}")

Number of Countries:174


In [406]:
## Drop unnessesary columns
columns_to_drop = ['Area Abbreviation','Area Code', 'Item Code', 'Element Code', 'Unit','latitude','longitude']
df.drop(columns_to_drop, axis=1, inplace = True)

# Check the outcome
df.columns

Index(['Area', 'Item', 'Element', 'Y1961', 'Y1962', 'Y1963', 'Y1964', 'Y1965',
       'Y1966', 'Y1967', 'Y1968', 'Y1969', 'Y1970', 'Y1971', 'Y1972', 'Y1973',
       'Y1974', 'Y1975', 'Y1976', 'Y1977', 'Y1978', 'Y1979', 'Y1980', 'Y1981',
       'Y1982', 'Y1983', 'Y1984', 'Y1985', 'Y1986', 'Y1987', 'Y1988', 'Y1989',
       'Y1990', 'Y1991', 'Y1992', 'Y1993', 'Y1994', 'Y1995', 'Y1996', 'Y1997',
       'Y1998', 'Y1999', 'Y2000', 'Y2001', 'Y2002', 'Y2003', 'Y2004', 'Y2005',
       'Y2006', 'Y2007', 'Y2008', 'Y2009', 'Y2010', 'Y2011', 'Y2012', 'Y2013'],
      dtype='object')

In [407]:
## Year columns showing the quantity of food and feed consumption to be melted
columns_to_keep = ['Area','Item','Element']
df = pd.melt(df, id_vars=columns_to_keep, var_name="Year", value_name="Weight (1000 tons)")

# Check the outcome
df.head()

,Area,Item,Element,Year,Weight (1000 tons)
0,Afghanistan,Wheat and products,Food,Y1961,1928.0
1,Afghanistan,Rice (Milled Equivalent),Food,Y1961,183.0
2,Afghanistan,Barley and products,Feed,Y1961,76.0
3,Afghanistan,Barley and products,Food,Y1961,237.0
4,Afghanistan,Maize and products,Feed,Y1961,210.0


In [408]:
## Strip 'Y' from the year
df['Year'] = df['Year'].str.strip('Y').astype(int)

# Check the outcome
df['Year'].head(2)

0    1961
1    1961
Name: Year, dtype: int32

In [409]:
## Group food and feed items into categories

df['Item'].nunique()

115

In [410]:
# Create lists of groups based on http://www.fao.org/nutrition/education/food-dietary-guidelines/en/
grains = ['Wheat and products','Rice (Milled Equivalent)','Barley and products','Maize and products',
          'Millet and products','Cereals, Other','Cereals - Excluding Beer','Rye and products','Oats',
          'Sorghum and products','Cassava and products']

fruits = ['Olives (including preserved)','Tomatoes and products','Oranges, Mandarines','Citrus, Other',
          'Bananas','Apples and products','Pineapples and products','Dates','Grapes and products (excl wine)',
          'Fruits, Other','Fruits - Excluding Wine','Lemons, Limes and products',
             'Grapefruit and products','Sugar cane']

vegetables = ['Potatoes and products','Vegetables, Other','Starchy Roots','Vegetables','Sweet potatoes',
              'Roots, Other','Onions','Plantains','Pimento','Aquatic Plants','Yams','Sugar beet']

sugars_sweeteners = ['Sugar (Raw Equivalent)','Sweeteners, Other','Sugar Crops',
                      'Sugar & Sweeteners','Sugar non-centrifugal']

legumes = ['Pulses, Other and products','Coffee and products','Cocoa Beans and products','Pulses',
          'Beans','Peas','Soyabeans','Groundnuts (Shelled Eq)']

nuts_seeds = ['Nuts and products','Coconuts - Incl Copra','Sesame seed','Treenuts','Rape and Mustardseed',
              'Palm kernels','Sunflower seed','Cottonseed']

oils = ['Soyabean Oil','Groundnut Oil','Sunflowerseed Oil','Rape and Mustard Oil','Cottonseed Oil',
        'Palm Oil','Sesameseed Oil','Olive Oil','Oilcrops Oil, Other','Oilcrops','Vegetable Oils',
        'Oilcrops, Other','Maize Germ Oil','Coconut Oil','Palmkernel Oil','Ricebran Oil']

beverages = ['Tea (including mate)','Wine','Beer','Beverages, Alcoholic',
             'Alcoholic Beverages','Beverages, Fermented']

spices = ['Pepper','Spices, Other','Spices','Cloves']

meat = ['Bovine Meat','Mutton & Goat Meat','Poultry Meat','Meat, Other','Meat','Pigmeat']

sea_food = ['Freshwater Fish','Fish, Seafood','Demersal Fish','Pelagic Fish','Marine Fish, Other','Crustaceans',
                'Cephalopods','Molluscs, Other','Aquatic Animals, Others','Aquatic Products, Other','Meat, Aquatic Mammals']

dairy = ['Butter, Ghee','Cream','Eggs','Milk - Excluding Butter','Infant food']

other_animal_products = ['Honey','Fish, Body Oil','Fish, Liver Oil','Offals','Offals, Edible','Animal fats','Fats, Animals, Raw']

misc = ['Stimulants','Miscellaneous']

In [411]:
## Create a column Category
df['Category'] = np.repeat(np.nan,df.shape[0])
df.head()

,Area,Item,Element,Year,Weight (1000 tons),Category
0,Afghanistan,Wheat and products,Food,1961,1928.0,NaN
1,Afghanistan,Rice (Milled Equivalent),Food,1961,183.0,NaN
2,Afghanistan,Barley and products,Feed,1961,76.0,NaN
3,Afghanistan,Barley and products,Food,1961,237.0,NaN
4,Afghanistan,Maize and products,Feed,1961,210.0,NaN


In [412]:
food_dict = {'Grains':grains,
             'Fruits':fruits, 
             'Vegetables':vegetables, 
             'Sugars_sweeteners':sugars_sweeteners,
             'Legumes':legumes, 
             'Nuts_seeds':nuts_seeds, 
            'Oils':oils, 
            'Beverages':beverages, 
            'Spices':spices, 
            'Meat':meat, 
            'Sea_food':sea_food, 
            'Dairy':dairy, 
            'Other_animal_products':other_animal_products, 
            'Miscellaneous':misc }

In [413]:
for key,value in food_dict.items():
    mask = df['Item'].isin(food_dict[key])
    df.loc[mask,'Category']  = key

In [414]:
df['Category'].value_counts()

Grains                   153806
Oils                     131387
Fruits                   125663
Vegetables               119621
Sea_food                 112625
Legumes                   87450
Other_animal_products     76903
Dairy                     75154
Meat                      56657
Beverages                 54166
Nuts_seeds                52629
Sugars_sweeteners         39856
Spices                    34450
Miscellaneous             17914
Name: Category, dtype: int64

In [415]:
# Final look
df.head()

,Area,Item,Element,Year,Weight (1000 tons),Category
0,Afghanistan,Wheat and products,Food,1961,1928.0,Grains
1,Afghanistan,Rice (Milled Equivalent),Food,1961,183.0,Grains
2,Afghanistan,Barley and products,Feed,1961,76.0,Grains
3,Afghanistan,Barley and products,Food,1961,237.0,Grains
4,Afghanistan,Maize and products,Feed,1961,210.0,Grains


# Additional information on world population

In [416]:
# I'm addding one more variable - Population
df1 = pd.read_csv(r'C:\Projects\World_food_consumption_trends-master\FoodBalanceSheets_E_All_Data_(Normalized).csv', encoding = 'latin1')

In [417]:
df1.dtypes

Area Code         int64
Area             object
Item Code         int64
Item             object
Element Code      int64
Element          object
Year Code         int64
Year              int64
Unit             object
Value           float64
Flag             object
dtype: object

In [418]:
df1.head()

,Area Code,Area,Item Code,Item,Element Code,Element,Year Code,Year,Unit,Value,Flag
0,2,Afghanistan,2501,Population,511,Total Population - Both sexes,1961,1961,1000 persons,8954.0,NaN
1,2,Afghanistan,2501,Population,511,Total Population - Both sexes,1962,1962,1000 persons,9142.0,NaN
2,2,Afghanistan,2501,Population,511,Total Population - Both sexes,1963,1963,1000 persons,9340.0,NaN
3,2,Afghanistan,2501,Population,511,Total Population - Both sexes,1964,1964,1000 persons,9547.0,NaN
4,2,Afghanistan,2501,Population,511,Total Population - Both sexes,1965,1965,1000 persons,9765.0,NaN


In [419]:
# Strip the data to leave only values relating to population
df1 = df1.loc[df1['Item'] == 'Population',:]

In [420]:
# Check the size of dataframe
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10393 entries, 0 to 11406278
Data columns (total 11 columns):
Area Code       10393 non-null int64
Area            10393 non-null object
Item Code       10393 non-null int64
Item            10393 non-null object
Element Code    10393 non-null int64
Element         10393 non-null object
Year Code       10393 non-null int64
Year            10393 non-null int64
Unit            10393 non-null object
Value           10393 non-null float64
Flag            2054 non-null object
dtypes: float64(1), int64(5), object(5)
memory usage: 974.3+ KB


In [421]:
# Drop unnessesary columns, we only want population values
columns_to_drop = ['Area Code', 'Item Code', 'Item', 'Element Code','Element','Year Code','Unit','Flag']
df1.drop(columns_to_drop, axis=1, inplace = True)

In [422]:
# Mere the two dataframes to reflect the population and store them in a new dataframe
df3 = pd.merge(df, df1, on=['Area','Year'], how = 'left')

In [423]:
# Rename Values to Population
df3.rename(columns={'Area':'Country','Value': 'Population (1000 ppl)'}, inplace=True)

In [424]:
# Final look
df3.head()

,Country,Item,Element,Year,Weight (1000 tons),Category,Population (1000 ppl)
0,Afghanistan,Wheat and products,Food,1961,1928.0,Grains,8954.0
1,Afghanistan,Rice (Milled Equivalent),Food,1961,183.0,Grains,8954.0
2,Afghanistan,Barley and products,Feed,1961,76.0,Grains,8954.0
3,Afghanistan,Barley and products,Food,1961,237.0,Grains,8954.0
4,Afghanistan,Maize and products,Feed,1961,210.0,Grains,8954.0


In [425]:
# Saving the result
df3.to_csv('FAO_master_population.csv', encoding='latin1')